In [ ]:
########################################################################
#####                   DREAM - MPPC Recording                     #####
########################################################################

In [1]:
##### Import Libraries
import numpy as np
import time
import os
import pickle
import matplotlib.pyplot as plt
import datetime
from mcculw import ul
from ctypes import cast, POINTER, c_double, c_ushort, c_ulong 
from mcculw.enums import (ScanOptions, FunctionType, Status, ChannelType, 
    DigitalPortType, ULRange, TriggerSource, TriggerSensitivity, TriggerEvent, 
    DigitalIODirection, TimerIdleState, PulseOutOptions, TrigType)
from mcculw.device_info import DaqDeviceInfo

try:
    from console_examples_util import config_first_detected_device
except ImportError:
    from console_examples_util import config_first_detected_device

## AQUISITION SETTINGS

In [2]:
DAQ_Rate = 10000 # [Hz]
Exp_Duration = 30 #[s]
EXPNAME = "RANDOM_92_2"

## SETUP DAQ

In [3]:
########################################################
##### SETUP EQUIPMENT

##### Connect to the DAQ Card
board_num = 0
daq_dev_info = DaqDeviceInfo(board_num = 0)

##### Setup DAQ for analog input
ai_info = daq_dev_info.get_ai_info()
ai_range = ai_info.supported_ranges[2]

## Set the scan options
scan_options_AI = ScanOptions.BACKGROUND

## Allocate a buffer for the input
memhandle_AI = ul.win_buf_alloc(int (DAQ_Rate*Exp_Duration))
if not memhandle_AI:
    raise Exception('Error: Failed to allocate memory')

## Convert the memhandle to a ctypes array
ctypes_array_AI = cast(memhandle_AI, POINTER(c_ushort))

## RUN EXPERIMENT

In [4]:
## Start the timer, which will start and pace AO and AI
ul.a_in_scan(board_num, 0, 0, (int (DAQ_Rate*Exp_Duration)), DAQ_Rate, ai_range, memhandle_AI, scan_options_AI)

## Wait until AI Finishes
status, curr_count, curr_index = ul.get_status(board_num, FunctionType.AIFUNCTION)
while status != Status.IDLE:
    time.sleep(0.5)
    status, curr_count, curr_index = ul.get_status(
            board_num, FunctionType.AIFUNCTION)

## READ AND SAVE ANALOG INPUT

In [5]:
points = []
for i in range((int (DAQ_Rate*Exp_Duration))):
    eng_value = ul.to_eng_units(board_num, ai_range, ctypes_array_AI[i])
    points.append(eng_value)
DAQ_signal = np.array(points)

In [6]:
DataFolderName = "C:/Users/Ian Coghill/OneDrive/ENS/Experiments/Harmonics/Data/" + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + "_" + str(EXPNAME)
os.makedirs(DataFolderName)
pickle.dump(DAQ_signal, open(DataFolderName + "/MPPCSignal.p", "wb" ))

In [7]:
## Display the data recorded
%matplotlib
plt.figure()
plt.xlabel('Sample')
plt.ylabel('Mean Signal')
plt.plot(DAQ_signal, "ko", markersize=0.5)
plt.show()

Using matplotlib backend: <object object at 0x00000210E94A0900>


In [8]:
## Display the data recorded
%matplotlib
plt.figure()
plt.xlabel('Sample')
plt.ylabel('Mean Signal')
plt.plot(DAQ_signal, "ko", markersize=0.5)
plt.show()

Using matplotlib backend: QtAgg
